In [1]:
import tensorflow as tf
import numpy as np
import math
import pandas as pd

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model
from keras.models import model_from_json
from tensorflow import keras

Using TensorFlow backend.


In [2]:
def xyPart(data):
    Xdata = data.iloc[:,0:177]
    Ydata = []
    for elem in data.iloc[:,177]:
        if elem == "Low":
            Ydata.append(0)
        elif elem == "High":
            Ydata.append(1)
    Ydata = np.array(Ydata)
    return Xdata, Ydata

In [4]:
# Load and preprocess data
data = pd.read_csv('HLM\HLMdesc.txt', sep="\t")
data = data.drop(columns = ['Protocol Number', 'Experiment Start Time', 'Experiment Stop Time', 'Result Type Name', 'Result Operator', 'Result Number Rounded',\
 'Project NumName', 'Smiles', 'Leonumber', 'DResult Text', 'DResult Text_1'])
data = data.drop(columns = ['Protocol Number_1', 'Experiment Start Time_1', 'Experiment Stop Time_1', 'Result Type Name_1', 'Result Operator_1', 'Result Number Rounded_1', 'Project NumName_1',\
       'Leonumber_1', 'origSmiles_1', 'ClappClass_1'])
print(data.columns)


Index(['rdk.fr_C_O_noCOO', 'rdk.Chi4v', 'rdk.fr_Ar_COO', 'rdk.fr_SH',
       'rdk.Chi4n', 'rdk.SMR_VSA10', 'rdk.fr_para_hydroxylation',
       'rdk.fr_barbitur', 'rdk.fr_halogen', 'rdk.fr_dihydropyridine',
       ...
       'rdk.PEOE_VSA9', 'rdk.fr_aldehyde', 'rdk.fr_pyridine',
       'rdk.fr_tetrazole', 'rdk.RingCount', 'rdk.fr_nitro_arom_nonortho',
       'rdk.Chi0v', 'rdk.fr_ArN', 'rdk.NumRotatableBonds', 'c#ClappClass'],
      dtype='object', length=178)


In [5]:
# Seperate test set
train = data.sample(frac=0.8,random_state=0)
test = data.drop(train.index)
#print(len(train))
#print(len(test))
Xtrain, Ytrain = xyPart(train)
Xtest, Ytest = xyPart(test)
print(Xtrain.columns)

Index(['rdk.fr_C_O_noCOO', 'rdk.Chi4v', 'rdk.fr_Ar_COO', 'rdk.fr_SH',
       'rdk.Chi4n', 'rdk.SMR_VSA10', 'rdk.fr_para_hydroxylation',
       'rdk.fr_barbitur', 'rdk.fr_halogen', 'rdk.fr_dihydropyridine',
       ...
       'rdk.MolMR', 'rdk.PEOE_VSA9', 'rdk.fr_aldehyde', 'rdk.fr_pyridine',
       'rdk.fr_tetrazole', 'rdk.RingCount', 'rdk.fr_nitro_arom_nonortho',
       'rdk.Chi0v', 'rdk.fr_ArN', 'rdk.NumRotatableBonds'],
      dtype='object', length=177)


In [6]:
# Normalize
data_stats = Xtrain.describe()
data_stats = data_stats.transpose()
def norm(x):
  return (x - data_stats['mean']) / data_stats['std']
nXtrain = norm(Xtrain)
nXtest = norm(Xtest)
#print(Xtrain.values[0])
#print(nXtrain.values[0])

In [7]:
# Rm columns containing NA
allCol = nXtrain.columns
ncXtrain = nXtrain.dropna(axis='columns')
col = ncXtrain.columns  # After removing NA cols
delCol = []
for c in allCol:
    if c not in col:
        delCol.append(c)
ncXtest = nXtest.drop(delCol, axis=1)
#print(nXtrain.values[0])
#print(ncXtrain.values[0])
#print(ncXtest.columns)

In [8]:
from sklearn.linear_model import LogisticRegression

learner = LogisticRegression()
model = learner.fit(ncXtrain, Ytrain)
print(model)

C:\Users\jgwdk\AppData\Local\Continuum\miniconda3\envs\myAzureEnv\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)


In [9]:
pred = model.predict(ncXtest)

In [10]:
def getCA(pred, Ytest):
    true = false = 0
    for idx in range(len(pred)):
        if Ytest[idx] == pred[idx]:
            true = true + 1
        else:
            false = false + 1
    print(true, false)        
    print("CA ", true/float(true+false))
    CA = true/float(true+false)
    return CA


In [11]:
print("Original model")
getCA(pred, Ytest)

import pickle
fid = open("modelDump.pkl", "wb")
modelFid = pickle.dump(model, fid)
fid.close()
#modelFid = pickle.dumps(model)


Original model
1222 259
CA  0.8251181634031061


In [12]:
fid = open("modelDump.pkl", "rb")
model2 = pickle.load(fid)
fid.close()
#model2 = pickle.loads(modelFid)
pred2 = model2.predict(ncXtest)
print("Loaded model")
getCA(pred2, Ytest)

Loaded model
1222 259
CA  0.8251181634031061


0.8251181634031061

In [13]:
import azureml.core
from azureml.core import Workspace

In [14]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

Found the config file in: C:\Users\jgwdk\Projects\AzureTutorial\aml_config\config.json
ML_ws_jgwdk	westeurope	euw-ml-test-rg	westeurope


In [31]:
from azureml.core import Experiment
exp = Experiment(workspace=ws, name="hlmexp")

In [23]:
print(exp.get_portal_url())
print(exp.identity)
print(exp.EXPERIMENT_PATH)
print(exp.name)
print(exp.from_directory)

https://mlworkspace.azure.ai/portal/subscriptions/f7ae3eeb-3ca0-42a2-b9e2-34c2480a9c1b/resourceGroups/euw-ml-test-rg/providers/Microsoft.MachineLearningServices/workspaces/ML_ws_jgwdk/experiments/HLMexp
Experiment#HLMexp
/experiments/{0}
HLMexp
<function Experiment.from_directory at 0x000001DC73FD1598>


In [21]:
print(dir(exp))

['DELIM', 'EXPERIMENT_PATH', 'PORTAL_URL', 'WORKSPACE_FMT', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_experiment_url', '_get_base_info_dict', '_identity', '_log_context', '_logger', '_name', '_prompt_message', '_repr_html_', '_serialize_to_dict', '_workspace', '_workspace_url', 'from_directory', 'get_docs_url', 'get_portal_url', 'get_runs', 'identity', 'list', 'name', 'start_logging', 'submit', 'workspace', 'workspace_object']


In [19]:
ws.get_details()

{'id': '/subscriptions/f7ae3eeb-3ca0-42a2-b9e2-34c2480a9c1b/resourceGroups/euw-ml-test-rg/providers/Microsoft.MachineLearningServices/workspaces/ML_ws_jgwdk',
 'name': 'ML_ws_jgwdk',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': '84aa8fd1-ddb1-45f3-ab24-51f645ae2097',
 'description': '',
 'friendlyName': '',
 'creationTime': '2019-01-31T08:52:13.8541198+00:00',
 'containerRegistry': '/subscriptions/f7ae3eeb-3ca0-42a2-b9e2-34c2480a9c1b/resourcegroups/euw-ml-test-rg/providers/microsoft.containerregistry/registries/mlwsjgwdk5062499165',
 'keyVault': '/subscriptions/f7ae3eeb-3ca0-42a2-b9e2-34c2480a9c1b/resourcegroups/euw-ml-test-rg/providers/microsoft.keyvault/vaults/mlwsjgwdk4598696655',
 'applicationInsights': '/subscriptions/f7ae3eeb-3ca0-42a2-b9e2-34c2480a9c1b/resourcegroups/euw-ml-test-rg/providers/microsoft.insights/components/mlwsjgwdk9665417812',
 'identityPrincipalId': 'e28dde4c-cff1-4736-94ff-053f7bd07c4f',
 'identityTenantId'

In [16]:
# Create resources

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = "AzureCluster"
compute_min_nodes = 0
compute_max_nodes = 4
# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = "STANDARD_D2_V2"


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-02-07T09:44:26.337000+00:00', 'creationTime': '2019-02-07T09:44:05.353545+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-02-07T09:44:40.612650+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [24]:
# Save model in Azure dir
os.makedirs('outputs', exist_ok=True)
fid = open("outputs\modelDump.pkl", "wb")
modelFid = pickle.dump(model, fid)
fid.close()

In [25]:
# Register model
from azureml.core.model import Model
model = Model.register(model_path = "outputs\modelDump.pkl",
                       model_name = "HLMmodel_v2",
                       tags = {"key": "0.1"},
                       description = "test registration",
                       workspace = ws)

Registering model HLMmodel_v2


In [ ]:
# import things required by this script
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

# load the model
def init():
    global model
    fid = open("outputs\modelDump.pkl", "rb")
    model = pickle.load(fid)
    fid.close()

# Passes data to the model and returns the prediction
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

In [27]:
# Create WS object
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {"data": "HLM", "type": "classification"}, 
                                               description = 'HLM')

In [26]:
# Create an image
from azureml.core.image import ContainerImage

# Image configuration
image_config = ContainerImage.image_configuration(execution_script = "scoreHLM.py",
                                                 runtime = "python",
                                                 conda_file = "myAzureEnv.yml",
                                                 description = "HLM class model",
                                                 tags = {"data": "HLM", "type": "class"}
                                                 )

In [30]:
from azureml.core.webservice import Webservice

# Create the webservice using all of the precreated configurations and our best model
service = Webservice.deploy_from_model(name='hlmwebservice',
                                       deployment_config = aciconfig,
                                       models=[model],
                                       image_config=image_config,
                                       workspace=ws)

# Wait for the service deployment to complete while displaying log output
service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image hlmwebservice:1, operation "Succeeded"
Creating service
Running........................
FailedACI service creation operation finished, operation "Failed"
Service creation polling reached terminal state, current service state: Unhealthy
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance hlmwebservice.\nYou can also try to run image mlwsjgwdk5062499165.azurecr.io/hlmwebservice:1 locally. Please refer to http://aka.ms/debugimage for more information."
    }
  ]
}


In [24]:
# Save data to test prediction
fid = open("ncXtest.pkl", "wb")
modelFid = pickle.dump(ncXtest, fid)
fid.close()

In [28]:
# Load test data
fid = open("ncXtest.pkl", "rb")
testData = pickle.load(fid)
fid.close()
#print(testData.values[0])